In [1]:
from langchain_together import ChatTogether

llm = ChatTogether(
    model="meta-llama/Llama-3-70b-chat-hf",
    together_api_key="your-together-api-key"
)


Unfortunately, I have to correct you - no human has stepped on Mars yet. While there have been numerous robotic missions to Mars, including rovers like NASA's Curiosity and Perseverance, as well as landers and orbiters from various countries and organizations, no human has traveled to Mars and set foot on its surface.

However, there are ongoing efforts to send humans to Mars in the coming decades, with NASA's Artemis program and SpaceX's Starship program being two of the most notable examples.


In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38954?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)


Apply for Director Technology, Japan

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyDirector Technology, JapanTokyo, JapanBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does 

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scrapped text is from the career's page of a website.
    Your job is to extract the jon postings and return them in JSON fromat containing the 
    following keys: 'role','experience','skills' and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)


    

```json
{
  "role": "Director Technology, Japan",
  "experience": "10+ years professional experience in technology related field",
  "skills": [
    "Bachelor’s degree in technology or equivalent",
    "Experience delivering and supporting consumer facing digital eCommerce, Retail technology products and experiences",
    "Expertise with end to end supply chain technology solutions and operations",
    "Experience leading or closely interfacing with technical operations teams, support service providers, and support organizations",
    "Technical product & technical solutions delivery experience",
    "Experience managing vendor relationships",
    "Team and performance management experience",
    "Excellent written and verbal communication skills; both English and Japanese (preferred)",
    "Strong influencing skills and ability to interact with senior management",
    "Experience delivering business and technology projects",
    "Deep understanding of technology management processes a

In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Director Technology, Japan',
 'experience': '10+ years professional experience in technology related field',
 'skills': ['Bachelor’s degree in technology or equivalent',
  'Experience delivering and supporting consumer facing digital eCommerce, Retail technology products and experiences',
  'Expertise with end to end supply chain technology solutions and operations',
  'Experience leading or closely interfacing with technical operations teams, support service providers, and support organizations',
  'Technical product & technical solutions delivery experience',
  'Experience managing vendor relationships',
  'Team and performance management experience',
  'Excellent written and verbal communication skills; both English and Japanese (preferred)',
  'Strong influencing skills and ability to interact with senior management',
  'Experience delivering business and technology projects',
  'Deep understanding of technology management processes and best practice operating models',
  

In [8]:
import pandas as pd
df = pd.read_csv("my_data.csv")
df

,Techstack,link
0,react javascript ai ml nodejs python,https://1-am-aditya.netlify.app/


In [18]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links":row["link"]},
                       ids=[str(uuid.uuid4())])
        

In [15]:
links = collection.query(query_texts=["Experience in Python", "Expertise in React"],n_results=1)
links
                         

{'ids': [['b2b89165-d8c8-4eb0-acd3-0eb3c8e0fabf'],
  ['b2b89165-d8c8-4eb0-acd3-0eb3c8e0fabf']],
 'distances': [[1.2702285710979442], [1.108971281514016]],
 'metadatas': [[{'links': 'https://1-am-aditya.netlify.app/'}],
  [{'links': 'https://1-am-aditya.netlify.app/'}]],
 'embeddings': None,
 'documents': [['react javascript ai ml nodejs python'],
  ['react javascript ai ml nodejs python']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [16]:
job = json_res
job['skills']

['Bachelor’s degree in technology or equivalent',
 'Experience delivering and supporting consumer facing digital eCommerce, Retail technology products and experiences',
 'Expertise with end to end supply chain technology solutions and operations',
 'Experience leading or closely interfacing with technical operations teams, support service providers, and support organizations',
 'Technical product & technical solutions delivery experience',
 'Experience managing vendor relationships',
 'Team and performance management experience',
 'Excellent written and verbal communication skills; both English and Japanese (preferred)',
 'Strong influencing skills and ability to interact with senior management',
 'Experience delivering business and technology projects',
 'Deep understanding of technology management processes and best practice operating models',
 'Experience with business process improvement methods (e.g., Lean) is preferred',
 'Understands agile methodology and is able to implement ag

In [20]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Technology Solutions for Your Japan Business Functions

Dear Hiring Manager,

I came across the job description for the Director Technology, Japan role and was impressed by the scope of responsibilities and the impact this position can have on your organization. As a Business Development Executive at AtliQ, I'd like to introduce you to our company's capabilities in delivering cutting-edge technology solutions that can support your Japan business functions.

At AtliQ, we specialize in AI & Software Consulting, empowering enterprises to streamline their business processes through automated tools. Our expertise in technical product & technical solutions delivery, vendor relationship management, and team performance management aligns perfectly with the requirements of the Director Technology, Japan role.

Our team has a proven track record of delivering business and technology projects, with a deep understanding of technology management processes and best practice operating